<a href="https://colab.research.google.com/github/petr0vsk/Fraud/blob/main/Fraud_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

кода для проверки процента ошибок по новой выгрузке

In [1]:
import pandas as pd

# 1. Загрузка новой выгрузки по транзакциям
def load_new_transactions(file_path):
    """
    Загружает новую выгрузку транзакций.

    Аргументы:
        file_path (str): путь к файлу с новыми транзакциями.

    Возвращает:
        pd.DataFrame: датафрейм с новыми транзакциями.
    """
    df = pd.read_csv(file_path, delimiter=';', header=None)
    df.columns = ['inn', 'org_name', 'amount', 'type', 'code', 'timestamp']

    # Преобразование суммы в числовой формат
    df['amount'] = df['amount'].apply(lambda x: str(x).replace(',', '.'))
    df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

    return df

# 2. Проверка новых транзакций с порогами кластеров
def check_new_transactions(new_transactions, cluster_files):
    """
    Проверяет новые транзакции по порогам срабатывания кластеров.

    Аргументы:
        new_transactions (pd.DataFrame): датафрейм с новыми транзакциями.
        cluster_files (list): список путей к файлам кластеров (Excel-файлы).

    Возвращает:
        pd.DataFrame: данные с результатами проверки по каждому кластеру.
    """
    all_results = pd.DataFrame()

    # Список для хранения информации по каждому кластеру
    cluster_info = []

    # Загрузка данных каждого кластера
    for cluster_file in cluster_files:
        cluster_name = cluster_file.split('_')[0]  # Извлечение имени кластера из имени файла
        df_cluster = pd.read_excel(cluster_file)

        # Сопоставление новых транзакций с кластерами по ИНН
        merged_df = pd.merge(new_transactions, df_cluster[['inn', 'threshold']], on='inn', how='left')

        # Проверка, превышают ли суммы пороги срабатывания
        merged_df['error'] = merged_df['amount'] > merged_df['threshold']

        # Подсчет ошибок и транзакций по каждому ИНН
        result_df = merged_df.groupby('inn').agg(
            total_transactions=('amount', 'count'),
            total_errors=('error', 'sum')
        ).reset_index()

        # Добавление данных по кластеру
        total_transactions = result_df['total_transactions'].sum()
        total_errors = result_df['total_errors'].sum()
        error_rate = (total_errors / total_transactions) * 100 if total_transactions > 0 else 0

        cluster_info.append({
            'Кластер': cluster_name,
            'Всего транзакций': total_transactions,
            'Ложных срабатываний': total_errors,
            'Процент ложных срабатываний': f"{error_rate:.2f}%"
        })

        # Добавляем в общие результаты
        all_results = pd.concat([all_results, result_df], ignore_index=True)

    return all_results, pd.DataFrame(cluster_info)

# 3. Подсчет процента ложных срабатываний
def calculate_error_rate(results_df):
    """
    Подсчитывает процент ложных срабатываний.

    Аргументы:
        results_df (pd.DataFrame): данные с результатами проверки.

    Возвращает:
        float: процент ложных срабатываний.
    """
    total_transactions = results_df['total_transactions'].sum()
    total_errors = results_df['total_errors'].sum()

    error_rate = (total_errors / total_transactions) * 100 if total_transactions > 0 else 0
    return error_rate

# 4. Основная часть программы
if __name__ == "__main__":
    # Загрузка новой выгрузки транзакций
    new_transactions_file = 'fraud_02.2024.csv'
    new_transactions = load_new_transactions(new_transactions_file)

    # Список файлов с данными по кластерам
    cluster_files = ['cluster_0_data_with_errors.xlsx', 'cluster_1_data_with_errors.xlsx', 'cluster_2_data_with_errors.xlsx']

    # Проверка новых транзакций
    results, cluster_info = check_new_transactions(new_transactions, cluster_files)

    # Подсчет процента ложных срабатываний по каждому кластеру
    print("\nИнформация по кластерам:")
    print(cluster_info.to_string(index=False))

    # Подсчет общего процента ложных срабатываний
    overall_error_rate = calculate_error_rate(results)
    print(f"\nОбщий процент ложных срабатываний по всем кластерам: {overall_error_rate:.2f}%")



Информация по кластерам:
Кластер  Всего транзакций  Ложных срабатываний Процент ложных срабатываний
cluster             86881                 1477                       1.70%
cluster             86881                  154                       0.18%
cluster             86881                   47                       0.05%

Общий процент ложных срабатываний по всем кластерам: 0.64%
